# From Lenses to Yoneda Embedding
###### (https://bartoszmilewski.com/2015/07/13/from-lenses-to-yoneda-embedding/)
### Plain functions
The most basic formulation of a lense is just a pair of functions

In [2]:
type Get s a = s -> a
type Set s a = s -> a -> s

We can define of polymorphic set that can change the type of the focused element, resulting in a new type for the containing object

In [4]:
type Set s t b = s -> b -> t

The lens must obey the following laws:
```
get . set s = id
get (set s a) = s
set (set s a) b = set s b
```

### Algebraic formulation
We can write the pair of functions as a single function by pulling the s:

In [5]:
type Combined s t a b = s -> (a, b -> t)

Going back to the monomorphic formulation, we write it using an ADT

In [18]:
data Store a s = Store a (a -> s)
type Lens s a = s -> Store a s

instance Functor (Store a) where
  fmap f (Store a s) = Store a (f . s)

We can further represent this a coalgebra

In [13]:
type Coalgebra w s = s -> w s
type Lens s a = Coalg (Store s) s

It's important to note that `Store` is a `Comonad`

In [14]:
class Comonad w where
  extract :: w a -> a
  duplicate :: w a -> w (w a)
  
instance Comonad (Store a) where
  extract (Store a f) = f a
  duplicate (Store a f) = Store a (\b -> Store b f)

Let's examine the compatibility of these two types, the Comonad and the Coalgebra

In [15]:
coalg :: Coalgebra (Store a) (a, b)
coalg (a, b) = Store a (\a' -> (a', b))

Since they go in different directions, they should cancel each other out

In [16]:
let tuple = (1, 2)
extract (coalg tuple) == tuple

True

Duplicating coalgebra should be the same applying coalgebra twice

In [20]:
(extract . extract) (duplicate (coalg tuple)) == (extract . extract) (fmap coalg (coalg tuple))

True

If these two conditions are satisfied, we call `coalg` a comonad coalgebra.
When applied to the `Store` comonad, these two conditions are equalivalent to the lens laws from earlier.

composing `extract` with `coalg` is the same as doing `set s (get s)`

For the other two laws, let's look at what happens when we `fmap` `coalg` onto `coalg`
```
\s -> Store (get s) ((\s' -> Store (get s') (set s')) . set s)
```
compared to what happens with `duplicate . coalg`
```
\s -> Store (get s) (\y -> Store y (set s))
```
So we want the second component from the `Store`s to be the same. The first one takes the result of `set s` and passes it to the function, giving us
```
Store (get (set s a)) (set (set s a))
```
compare to `Store y (set s)`, this gives us the `get (set s a) = a` and `set (set s a) b = set s b` laws.


The Algebraic structure can be modified to have the polymorphic lens behaviour by using the indexed version of `Store` (called `Context` in lens library) and the indexed version of Comonad

In [24]:
data IStore a b t = IStore a (b -> t)

class IxFunctor f where
  imap :: (s -> t) -> f a b s -> f a b t
  
instance IxFunctor IStore where
  imap f (IStore a h) = IStore a (f . h)
  
class IxComonad w where
  iextract :: w a a t -> t
  iduplicate :: w a b t -> w a j (w j b t)
  
instance IxComonad IStore where
  iextract (IStore a at) = at a
  iduplicate (IStore a bt) = IStore a (\j -> IStore j bt)
  
type ICoalg w s t a b = s -> w a b t

### Parametric functions
This standard representation of lenses are awkward to use because they don't compose well. Van Laarhoven discovered the following formulation

In [29]:
:ext RankNTypes
type Lens s t a b = forall f. Functor f => (a -> f b) -> (s -> f t)

It's a polymorphic function that is polymorphic over a functor type. To see how it's equivalent to the get/set one, start by pulling the `s` out of the forall quantifier

In [30]:
type Lens s t a b = s -> (forall f. Functor f => (a -> f b) -> f t)

This states that you can provide an s and a function that converts `a` to `b` under any functor and get back a `t` in that functor. Since the function must be polymorphic for any such functor, the only thing we can do with that functor is to use it's `fmap` implementation.

the `(a -> f b) -> f t` part implies that we have access to some `a` value in order to get the `f b` and also some function of type `b -> t` to fmap in and produce the final result. These two things are just the contents of the `IStore`, meaning it's equivalent to
```
\s -> IStore a b t
```
The choice of functors, `Const` for getting and `Identity` for setting, is how we recover the get/set from the Van Laarhoven construction.

The beauty of the Van Laarhoven representation is that it composes very nicely; It's a function that takes a function and returns a function - the returned function of one lens becomes the input function of the next so that they nest inside one another. `Lens s t a b` nests into `Lens x y s t`, etc.

### Categorical picture
There's a way to derive Van Laarhoven lenses using the Yoneda embedding, which embeds any category in the category of functors from that category to Set.
```
forall x. a -> C(a, x)
```
This is mapping from objects in `C` to functors which themselves are objects in the functor category.

So given an object `a` in category `C`, we have a functor from `C` to Set, specifically the hom-sets where `a` is the source object.

Since we have a mapping between objects in two different categories (`C` and the category of Functors) we should check if this mapping is itself a Functor, i.e. what is it's action on morphisms. A morphism in C is just an element of `C(a, b)` while a morhism in `[C, Set]` is a natural transformation, so we are looking for a mapping from morphisms to natural transformations. For this, we can use the Yoneda lemma
```
[C, Set](C(a, -), F) = F a
```
replacing the generic F with `C(b, -)`, we get
```
[C, Set](C(a, -), C(b, -)) = C(b, a)
```
The morphism C(b, a) is reverse because this is a contravariant functor.

We can look at this result as saying that in order to have a natural transformation between the two hom-functors `C(a, -)` and `C(b, -)`, we must have access to a morphism from `C(b, a)` that we can pre-compose with the `C(a, x)` morphism from the first hom-set, thus producing a `C(b, x)`. Any category can be embedded in the category of Functors in this way, and the underlying mapping is a fully faithful Functor from `C` to `[C, Set]`.

An interesting effect of this is that a Functor category can itself be embedded in the category of functors from that functor category to Set.

We can show this by fixing a Functor `R` in the hom-set to some arbitrary functor `f` which is a set of natural transformations between two functors, `Nat(R, f)`. We know that this mapping is itself a functor, this time a functor from a functor category to Set.
```
Nat(Nat(R, f), Nat(S, f)) = Nat(S, R)
```
Theory: This recursive application of the yoneda embedding is reminiscent of the nesting that we saw with Van Laarhoven lenses.

The outer natural transformation above is interesting in that it is a family of natural transformations paramterized by functors, whereas a normal natural transformation is a family of morphisms parameterized by objects.

We gain some intuition, we can translate these natural transformations to Haskell representations.
```
forall x. R x -> f x
forall y. S y -> f y
```
These are families of functions (morphisms) paramterized by a type (object). The outer natural transformation is paramterized by a Functor:
```
forall f. Functor f => (forall x. R x -> f x) -> (forall y. S y -> f y)
```
We can already see a similarity to Van Laarhoven lenses: the quantification over a functor.

The right hand side of this yoneda embedding is a natural transformation `forall z. S z -> R z`. The next step is to determine which functors to use for S and R - let's have R be `IStore a b` and S be `IStore s t`. Let's look at the first part of the left side:
```
forall x. IStore a b x -> f x
```
a function from a IStore is equivalent to a function from it's two component parts:
```
forall x. a -> (b -> x) -> f x
```
we can pull the a out of the forall quantifier to get
```
a -> (forall x. (b -> x) -> f x)
```
The thing in the parenthesis is a natural transformation from a hom-functor to some other functor, so let's use the yoneda lemma to see that `(b -> x) -> f x` is equivalent to `f b`.
```
a -> f b
```
That's the first part of the Van Laarhoven! Similarly, the right part can be translated from
```
forall y. IStore s t y -> f y
```
to
```
s -> f t
```
taken together we have
```
forall f. Functor f => (a -> f b) -> (s -> f t) = forall z. IStore s t z -> IStore a b z
```
now let's apply these same methods to the right hand side:
```
forall z. s -> (t -> z) -> IStore a b z

s -> (forall z. (t -> z) -> IStore a b z)

s -> IStore a b t
```
Thus we have arrived at the standard representation of a lens.